In [1]:
!pip install langchain
!pip install openai

import os
os.environ["OPENAI_API_KEY"] = "YOUR API KEY"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.3/241.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00


## Large Language Models (LLMs)

In [2]:
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.1 MB/s eta 0:00:00


In [4]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.5)
text = "What is a good name for a framework that makes large language models easier to work with?"
print(llm.invoke(text))



"LanguageFlow"


## Connect to Hugging Face Hub

In [ ]:
!pip install huggingface_hub > /dev/null

In [ ]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
from langchain_community.llms import HuggingFaceHub

repo_id = "google/flan-t5-base" # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":0, "max_length":64})
llm

HuggingFaceHub(client=<InferenceClient(model='google/flan-t5-base', timeout=None)>, repo_id='google/flan-t5-base', task='text2text-generation', model_kwargs={'temperature': 0, 'max_length': 64})

In [ ]:
text = "Please answer the following question. What is the boiling point of water in farenheit?"

llm(text)

'212'

# Chat Models

In [6]:
from langchain_openai import ChatOpenAI
from langchain.schema import (
    AIMessage, # AI input
    HumanMessage, # user input
    SystemMessage # sets the tone of the conversation
)

chat = ChatOpenAI(temperature=0,
                  max_tokens=100)

messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming.")
]
chat.invoke(messages)

AIMessage(content="J'adore la programmation.")

### Have A Conversation

In [8]:
chat = ChatOpenAI(temperature=0,
                  max_tokens=1000)

context = [SystemMessage(content="You are a friendly Chatbot that likes to have conversations.")]

while True:
    user_message = input("You: ")
    context.append(HumanMessage(content=user_message))
    if user_message.lower() == "quit":
        break
    response = chat.invoke(context)

    context.append(response)
    print("AI: ", response.content)

You: Tell me about AI
AI:  Artificial Intelligence (AI) is a branch of computer science that focuses on creating machines or systems that can perform tasks that typically require human intelligence. These tasks include things like learning, reasoning, problem-solving, perception, language understanding, and decision-making.

AI systems can be designed to perform specific tasks, such as playing chess or recognizing speech, or they can be more general-purpose and capable of learning and adapting to new situations. There are different approaches to AI, including machine learning, neural networks, natural language processing, and computer vision.

AI has a wide range of applications across various industries, including healthcare, finance, transportation, and entertainment. It has the potential to revolutionize how we work, live, and interact with technology in the future. However, there are also ethical and societal implications to consider as AI continues to advance.
You: quit


## Embedding Models

In [9]:
!pip install tiktoken

In [11]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

text = "This is the text you are going to embed"

query_result = embeddings.embed_query(text)

len(query_result)

1536

In [12]:
query_result[:10]

[-0.020759657972839456,
 0.017775294518687895,
 -0.009025084388266495,
 -0.0031921573438719176,
 -0.008887647259343948,
 0.029817466240967656,
 -0.02523620501671187,
 -0.022317287459638648,
 -0.029450964746980772,
 -0.00797139445569927]

### Using Embeddings to Distinguish Good & Bad Reviews

In [13]:
positive_reviews = [
    "A cinematic masterpiece that captivates from start to finish.",
    "Outstanding storytelling with breathtaking performances!",
    "Exquisitely crafted and emotionally resonant.",
    "An enthralling experience that stays with you.",
    "Exceptional film! A triumph in every aspect."
]

negative_reviews = [
    "A tedious experience, lacks creativity and depth.",
    "The story drags and never quite lands.",
    "Lackluster performances and dull script.",
    "Disappointingly shallow plot and weak characterization.",
    "Fails to deliver on its promise. A missed opportunity."
]

In [14]:
import numpy as np


positive_embeds, negative_embeds = [], []

for review in positive_reviews:
  embedding = embeddings.embed_query(review)
  positive_embeds.append(embedding)

for review in negative_reviews:
  embedding = embeddings.embed_query(review)
  negative_embeds.append(embedding)

In [15]:
positive_review = positive_reviews[0]
positive_review

'A cinematic masterpiece that captivates from start to finish.'

In [16]:
max_sim = 0

similarity_dict = {}

for i, embed in enumerate(positive_embeds):
  sim = np.dot(positive_embeds[0], embed)
  similarity_dict[f'p{i}'] = sim
  if max_sim < sim:
    max_sim = sim

for i, embed in enumerate(negative_embeds):
  sim = np.dot(positive_embeds[0], embed)
  similarity_dict[f'n{i}'] = sim
  if max_sim < sim:
    max_sim = sim

for similarity in similarity_dict:
  similarity_dict[similarity] = 100 * (similarity_dict[similarity] / max_sim)

similarity_dict

{'p0': 100.0,
 'p1': 87.13739141277141,
 'p2': 87.9873483799291,
 'p3': 87.5905139274744,
 'p4': 90.49079910414294,
 'n0': 78.94964001283236,
 'n1': 80.73916860122527,
 'n2': 80.45529929517066,
 'n3': 80.28967487753151,
 'n4': 78.72574387589792}